In [3]:
#Connect MongoDb and snowflake to load the data
import pandas as pd
from pymongo import MongoClient
import snowflake.connector

try:
    # Step 1: Connect to MongoDB
    mongo_client = MongoClient("mongodb://localhost:27017")  # Update if using Atlas
    db = mongo_client["NewDB"]
    collection = db["healthcare"]

    # Step 2: Load data from MongoDB
    documents = list(collection.find())
    if not documents:
        raise ValueError("No documents found in MongoDB collection.")

    # Step 3: Convert to DataFrame
    df = pd.DataFrame(documents)

    # Step 4: Drop MongoDB internal _id field
    if '_id' in df.columns:
        df = df.drop(columns=['_id'])

    # Step 5: Convert date columns to string
    for col in df.select_dtypes(include=['datetime', 'datetimetz', 'date']).columns:
        df[col] = df[col].astype(str)

    # Step 6: Connect to Snowflake
    conn = snowflake.connector.connect(
        user='Fizan',
        password='Tietoevry12345',
        account='bu51577.central-india.azure',
        warehouse='SNOWFLAKE_LEARNING_WH',
        database='SNOWFLAKE_LEARNING_DB',
        schema='MTM_ANALYTICS',
        role='ACCOUNTADMIN'
    )
    cursor = conn.cursor()

    # Step 7: Prepare Insert Query
    table_name = 'MTM_RECORDS'
    columns = list(df.columns)
    insert_query = f"""
        INSERT INTO {table_name} ({', '.join(columns)})
        VALUES ({', '.join(['%s'] * len(columns))})
    """

    # Step 8: Insert Data
    for _, row in df.iterrows():
        values = tuple(row.fillna("").values.tolist())
        cursor.execute(insert_query, values)

    conn.commit()
    print("✅ JSON data from MongoDB loaded into Snowflake.")

except Exception as e:
    print(f"❌ Error: {e}")

finally:
    try:
        cursor.close()
        conn.close()
    except:
        pass


❌ Error: localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 683584a7dc3ce87067aeedd4, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>


In [ ]:
#To get the MTM Data to display at the front end
import snowflake.connector
import pandas as pd
import json
import datetime

try:
    # Step 1: Connect to Snowflake
    conn = snowflake.connector.connect(
        user='Fizan',
        password='Tietoevry12345',
        account='bu51577.central-india.azure',
        warehouse='SNOWFLAKE_LEARNING_WH',
        database='SNOWFLAKE_LEARNING_DB',
        schema='MTM_ANALYTICS',
        role='ACCOUNTADMIN'
    )
    cursor = conn.cursor()

    # Step 2: Fetch all data from the table
    query = "SELECT * FROM MTM_RECORDS"
    cursor.execute(query)

    # Step 3: Get column names
    columns = [col[0] for col in cursor.description]

    # Step 4: Load rows into DataFrame
    rows = cursor.fetchall()
    df = pd.DataFrame(rows, columns=columns)

    # Step 5: Convert all datetime/date columns to string
    for col in df.columns:
        if pd.api.types.is_datetime64_any_dtype(df[col]):
            df[col] = df[col].astype(str)
        else:
            if df[col].dtype == 'object':
                if df[col].dropna().apply(lambda x: isinstance(x, (datetime.date, datetime.datetime))).all():
                    df[col] = df[col].astype(str)

    # Step 6: Convert to JSON
    json_data = df.to_dict(orient='records')

    # Step 7: Save to file
    with open('mtm_data_export.json', 'w', encoding='utf-8') as f:
        json.dump(json_data, f, ensure_ascii=False, indent=4)

    print("✅ Data exported to mtm_data_export.json successfully.")

except Exception as e:
    print(f"❌ Error: {e}")

finally:
    try:
        cursor.close()
        conn.close()
    except:
        pass


✅ Data exported to mtm_data_export.json successfully.


In [ ]:
#Script to connect python and Snowflake
import snowflake.connector
import pandas as pd

try:
    # Connect to Snowflake
    conn = snowflake.connector.connect(
        user='Fizan',
        password='Tietoevry12345',
        account='bu51577.central-india.azure',
        warehouse='SNOWFLAKE_LEARNING_WH',
        database='SNOWFLAKE_LEARNING_DB',
        schema='MTM_ANALYTICS',
        role='ACCOUNTADMIN'
    )
    cursor = conn.cursor()

    # Load data
    final_df = pd.read_csv("MTMData.csv")

    # Rename columns to match Snowflake table exactly
    final_df.rename(columns={
        "Record Type": "RECORD_TYPE",
        "Transaction ID": "TRANSACTION_ID",
        "Date": "DATE",
        "Pharmacy NCPDP ID": "PHARMACY_NCPDP_ID",
        "Pharmacist NPI": "PHARMACIST_NPI",
        "Patient ID": "PATIENT_ID",
        "Patient Name": "PATIENT_NAME",
        "DOB": "DOB",
        "Gender": "GENDER",
        "Payer ID": "PAYER_ID",
        "Plan Name": "PLAN_NAME",
        "Intervention Type": "INTERVENTION_TYPE",
        "MTM Service Code": "MTM_SERVICE_CODE",
        "Start Date": "START_DATE",
        "End Date": "END_DATE",
        "Outcome": "OUTCOME",
        "Recommendations": "RECOMMENDATIONS",
        "Prescriber Contacted": "PRESCRIBER_CONTACTED",
        "Prescriber NPI": "PRESCRIBER_NPI",
        "Prescriber Response": "PRESCRIBER_RESPONSE",
        "Follow-up Date": "FOLLOW_UP_DATE",
        "Notes": "NOTES"
    }, inplace=True)

    # Ensure all required columns exist
    expected_columns = [
        'RECORD_TYPE', 'TRANSACTION_ID', 'DATE', 'PHARMACY_NCPDP_ID', 'PHARMACIST_NPI',
        'PATIENT_ID', 'PATIENT_NAME', 'DOB', 'GENDER', 'PAYER_ID', 'PLAN_NAME',
        'INTERVENTION_TYPE', 'MTM_SERVICE_CODE', 'START_DATE', 'END_DATE',
        'OUTCOME', 'RECOMMENDATIONS', 'PRESCRIBER_CONTACTED', 'PRESCRIBER_NPI',
        'PRESCRIBER_RESPONSE', 'FOLLOW_UP_DATE', 'NOTES'
    ]
    missing_cols = [col for col in expected_columns if col not in final_df.columns]
    if missing_cols:
        raise ValueError(f"Missing columns in CSV: {missing_cols}")

    # Insert rows
    insert_query = f"""
        INSERT INTO MTM_RECORDS ({', '.join(expected_columns)})
        VALUES ({', '.join(['%s'] * len(expected_columns))})
    """
    for _, row in final_df.iterrows():
        values = tuple(row.fillna("").values.tolist())
        cursor.execute(insert_query, values)

    conn.commit()
    print("✅ Data uploaded successfully to MTM_RECORDS table.")


except Exception as e:
    print(f"❌ Error: {e}")

finally:
    try:
        cursor.close()
        conn.close()
    except:
        pass


✅ Data uploaded successfully to MTM_RECORDS table.


In [ ]:
import snowflake.connector

try:
    # Connect to Snowflake
    conn = snowflake.connector.connect(
        user='Fizan',
        password='',
        account='bu51577.central-india.azure',  # Include region for your Snowflake account
        warehouse='SNOWFLAKE_LEARNING_WH',
        database='SNOWFLAKE_LEARNING_DB',
        schema='MTM_ANALYTICS',
        role='ACCOUNTADMIN'
    )

    cursor = conn.cursor()
    cursor.execute("SELECT CURRENT_USER()")
    user = cursor.fetchone()
    print(f"Connected to Snowflake as user: {user[0]}")

    cursor.close()
    conn.close()

except Exception as e:
    print("Failed to connect to Snowflake:", e)


Connected to Snowflake as user: FIZAN


In [2]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 15.0 MB/s eta 0:00:00


In [1]:
!pip install snowflake-connector-python pandas



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.8/70.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 6.2 MB/s eta 0:00:00


In [ ]:
import csv
import json
import os

def csv_to_json_mtm(csv_dir, json_filepath, limit=50):
    patient_data = {}

    # Base structure
    csv_files = [
        'patients.csv', 'careplans.csv', 'providers.csv', 'allergies.csv', 'procedures.csv',
        'observations.csv', 'medications.csv', 'payers.csv', 'supplies.csv', 'conditions.csv',
        'devices.csv', 'encounters.csv', 'imaging_studies.csv', 'immunizations.csv',
        'organizations.csv', 'payer_transitions.csv'
    ]

    # Step 1: Load patients.csv
    try:
        with open(os.path.join(csv_dir, 'patients.csv'), 'r', encoding='utf-8') as file:
            for row in csv.DictReader(file):
                pid = row.get('Id')
                if pid:
                    patient_data[pid] = {
                        'patient_details': row,
                        'mtm': {
                            'interventions': [],
                            'medications': [],
                            'clinical_issues': [],
                            'outcomes': [],
                            'follow_up_actions': [],
                            'billing': []
                        },
                        'other_data': {}
                    }
    except Exception as e:
        print(f"Error reading patients.csv: {e}")
        return

    # Step 2: Process other CSVs
    for filename in csv_files[1:]:
        path = os.path.join(csv_dir, filename)
        try:
            with open(path, 'r', encoding='utf-8') as file:
                reader = csv.DictReader(file)
                count = 0
                for row in reader:
                    if count >= limit:
                        break
                    pid = row.get('PATIENT')
                    if not pid or pid not in patient_data:
                        continue

                    # MTM Mappings
                    mtm = patient_data[pid]['mtm']
                    if filename in ['careplans.csv', 'procedures.csv']:
                        mtm['interventions'].append(row)
                    elif filename == 'medications.csv':
                        mtm['medications'].append(row)
                    elif filename in ['conditions.csv', 'observations.csv']:
                        mtm['clinical_issues'].append(row)
                    elif filename in ['observations.csv', 'encounters.csv']:
                        mtm['outcomes'].append(row)
                    elif filename in ['careplans.csv', 'encounters.csv']:
                        mtm['follow_up_actions'].append(row)
                    elif filename in ['payers.csv', 'payer_transitions.csv']:
                        mtm['billing'].append(row)
                    else:
                        key = filename.replace('.csv', '')
                        if key not in patient_data[pid]['other_data']:
                            patient_data[pid]['other_data'][key] = []
                        patient_data[pid]['other_data'][key].append(row)

                    count += 1
        except FileNotFoundError:
            print(f"File not found: {filename}")
        except Exception as e:
            print(f"Error reading {filename}: {e}")

    # Step 3: Write to JSON
    try:
        with open(json_filepath, 'w', encoding='utf-8') as out:
            json.dump(patient_data, out, indent=4)
        print(f"✅ MTM-formatted JSON saved to {json_filepath}")
    except Exception as e:
        print(f"Error writing JSON: {e}")

# Usage
csv_to_json_mtm('drive/MyDrive/NCPDP/csv', 'mtm_patient_output.json')


✅ MTM-formatted JSON saved to mtm_patient_output.json


In [ ]:
import csv
import json
import os

def csv_to_json_nested(csv_dir, json_filepath, limit=50):
    files = [
        'patients.csv', 'careplans.csv', 'providers.csv', 'allergies.csv',
        'procedures.csv', 'observations.csv', 'medications.csv', 'payers.csv',
        'supplies.csv', 'conditions.csv', 'devices.csv', 'encounters.csv',
        'imaging_studies.csv', 'immunizations.csv', 'organizations.csv',
        'payer_transitions.csv'
    ]

    patient_data = {}

    # Step 1: Load patients.csv
    try:
        with open(os.path.join(csv_dir, 'patients.csv'), 'r', encoding='utf-8') as f:
            for row in csv.DictReader(f):
                pid = row.get('Id')
                if pid:
                    patient_data[pid] = {'patient_details': row}
    except Exception as e:
        print(f"Error loading patients.csv: {e}")
        return

    # Step 2: Load all other files and group by PATIENT ID
    for file in files[1:]:
        path = os.path.join(csv_dir, file)
        key = file.replace('.csv', '')

        try:
            with open(path, 'r', encoding='utf-8') as f:
                for i, row in enumerate(csv.DictReader(f)):
                    if i >= limit:
                        break
                    pid = row.get('PATIENT')
                    if pid in patient_data:
                        patient_data[pid].setdefault(key, []).append(row)
        except FileNotFoundError:
            print(f"Missing file: {file}, skipping...")
        except Exception as e:
            print(f"Error reading {file}: {e}")

    # Step 3: Save to JSON
    try:
        with open(json_filepath, 'w', encoding='utf-8') as f:
            json.dump(patient_data, f, indent=4)
        print(f"✅ Data saved to {json_filepath}")
    except Exception as e:
        print(f"Error writing JSON: {e}")

# Example usage
csv_dir = 'drive/MyDrive/NCPDP/csv'
json_filepath = 'sampleOutput.json'
csv_to_json_nested(csv_dir, json_filepath)


✅ Data saved to sampleOutput.json


In [ ]:
import csv
import json
import os

def csv_to_json_nested(csv_dir, json_filepath, limit=50):
    patient_data = {}  # Dictionary to store patient data

    csv_files = ['patients.csv', 'careplans.csv',
                 'providers.csv', 'allergies.csv',
                 'procedures.csv', 'observations.csv',
                 'medications.csv','payers.csv',
                 'supplies.csv','conditions.csv',
                 'devices.csv','encounters.csv',
                 'imaging_studies.csv','immunizations.csv',
                 'organizations.csv','payer_transitions.csv']  # Add your CSV file names here

    # Process patients.csv first to create the root structure
    patients_filepath = os.path.join(csv_dir, 'patients.csv')
    try:
        with open(patients_filepath, 'r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                patient_id = row.get('Id')  # Get patient ID from 'id' column
                if patient_id:
                    patient_data[patient_id] = {'patient_details': row}  # Store patient details as root
    except FileNotFoundError:
        print(f"File not found: {patients_filepath}")
        return
    except Exception as e:
        print(f"An error occurred while reading {patients_filepath}: {e}")
        return

    # Process other CSV files and nest data under patient IDs
    for filename in csv_files[1:]:  # Skip patients.csv (already processed)
        csv_filepath = os.path.join(csv_dir, filename)
        print(f"Checking file: {csv_filepath}")

        try:
            with open(csv_filepath, 'r', encoding='utf-8') as csvfile:
                reader = csv.DictReader(csvfile)
                print(f"Reading file: {csv_filepath}")
                count = 0
                for row in reader:
                    if count < limit:
                        patient_id = row.get('PATIENT')  # Get patient ID from 'PATIENT' column
                        if patient_id and patient_id in patient_data:
                            # Use filename as key for nested data
                            if filename[:-4] not in patient_data[patient_id]:
                                patient_data[patient_id][filename[:-4]] = []
                            patient_data[patient_id][filename[:-4]].append(row)
                            count += 1
                            print(f"Added data for patient {patient_id} from {filename}")
                    else:
                        break
        except FileNotFoundError:
            print(f"File not found: {csv_filepath}")
            continue
        except Exception as e:
            print(f"An error occurred while reading {csv_filepath}: {e}")
            return

    print(f"Total patients: {len(patient_data)}")

    try:
        with open(json_filepath, 'w', encoding='utf-8') as jsonfile:
            json.dump(patient_data, jsonfile, indent=4)
        print(f"Successfully converted data from multiple CSV files to nested JSON.")
    except Exception as e:
        print(f"An error occurred while writing to JSON file: {e}")

# Example usage:
csv_dir = 'drive/MyDrive/NCPDP/csv'  # Directory containing CSV files
json_filepath = 'sampleOutput.json'
csv_to_json_nested(csv_dir, json_filepath)

Checking file: drive/MyDrive/NCPDP/csv/careplans.csv
Reading file: drive/MyDrive/NCPDP/csv/careplans.csv
Added data for patient 8b0484cd-3dbd-8b8d-1b72-a32f74a5a846 from careplans.csv
Added data for patient 8b0484cd-3dbd-8b8d-1b72-a32f74a5a846 from careplans.csv
Added data for patient b8eb8d31-1031-fb5b-e207-b9815f80744c from careplans.csv
Added data for patient ce9bd436-6b59-0452-86a4-61f3642736bc from careplans.csv
Added data for patient ce9bd436-6b59-0452-86a4-61f3642736bc from careplans.csv
Added data for patient ce9bd436-6b59-0452-86a4-61f3642736bc from careplans.csv
Added data for patient b8eb8d31-1031-fb5b-e207-b9815f80744c from careplans.csv
Added data for patient 6fc3e360-ae68-c411-e091-4734df51eb18 from careplans.csv
Added data for patient 6fc3e360-ae68-c411-e091-4734df51eb18 from careplans.csv
Added data for patient ce4ce4d8-d4e2-aca2-5a92-8ce703c5077a from careplans.csv
Added data for patient ce4ce4d8-d4e2-aca2-5a92-8ce703c5077a from careplans.csv
Added data for patient ce4

In [ ]:
# @title Default title text
import csv
import json
import os

def csv_to_json_nested(csv_dir, json_filepath):  # Removed limit parameter
    patient_data = {}  # Dictionary to store patient data

    csv_files = ['patients.csv', 'careplans.csv',
                 'providers.csv', 'allergies.csv',
                 'procedures.csv', 'observations.csv',
                 'medications.csv','payers.csv',
                 'supplies.csv','conditions.csv',
                 'devices.csv','encounters.csv',
                 'imaging_studies.csv','immunizations.csv',
                 'organizations.csv','payer_transitions.csv']  # Add your CSV file names here

    # Process patients.csv first to create the root structure
    patients_filepath = os.path.join(csv_dir, 'patients.csv')
    try:
        with open(patients_filepath, 'r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                patient_id = row.get('Id')  # Get patient ID from 'id' column
                if patient_id:
                    patient_data[patient_id] = {'patient_details': row}  # Store patient details as root
    except FileNotFoundError:
        print(f"File not found: {patients_filepath}")
        return
    except Exception as e:
        print(f"An error occurred while reading {patients_filepath}: {e}")
        return

    # Process other CSV files and nest data under patient IDs
    for filename in csv_files[1:]:  # Skip patients.csv (already processed)
        csv_filepath = os.path.join(csv_dir, filename)
        print(f"Checking file: {csv_filepath}")

        try:
            with open(csv_filepath, 'r', encoding='utf-8') as csvfile:
                reader = csv.DictReader(csvfile)
                print(f"Reading file: {csv_filepath}")

                for row in reader:  # Removed count and limit check
                    patient_id = row.get('PATIENT')  # Get patient ID from 'PATIENT' column
                    if patient_id and patient_id in patient_data:
                        # Use filename as key for nested data
                        if filename[:-4] not in patient_data[patient_id]:
                            patient_data[patient_id][filename[:-4]] = []
                        patient_data[patient_id][filename[:-4]].append(row)

                        print(f"Added data for patient {patient_id} from {filename}")

        except FileNotFoundError:
            print(f"File not found: {csv_filepath}")
            continue
        except Exception as e:
            print(f"An error occurred while reading {csv_filepath}: {e}")
            return

    print(f"Total patients: {len(patient_data)}")

    try:
        with open(json_filepath, 'w', encoding='utf-8') as jsonfile:
            json.dump(patient_data, jsonfile, indent=4)
        print(f"Successfully converted data from multiple CSV files to nested JSON.")
    except Exception as e:
        print(f"An error occurred while writing to JSON file: {e}")

# Example usage:
csv_dir = 'drive/MyDrive/NCPDP/csv'  # Directory containing CSV files
json_filepath = 'output.json'
csv_to_json_nested(csv_dir, json_filepath)

Streaming output truncated to the last 5000 lines.
Added data for patient 19ac1829-945a-c4a3-63fb-ce50c89977ca from medications.csv
Added data for patient 19ac1829-945a-c4a3-63fb-ce50c89977ca from medications.csv
Added data for patient 19ac1829-945a-c4a3-63fb-ce50c89977ca from medications.csv
Added data for patient 19ac1829-945a-c4a3-63fb-ce50c89977ca from medications.csv
Added data for patient 19ac1829-945a-c4a3-63fb-ce50c89977ca from medications.csv
Added data for patient 19ac1829-945a-c4a3-63fb-ce50c89977ca from medications.csv
Added data for patient 19ac1829-945a-c4a3-63fb-ce50c89977ca from medications.csv
Added data for patient 19ac1829-945a-c4a3-63fb-ce50c89977ca from medications.csv
Added data for patient 19ac1829-945a-c4a3-63fb-ce50c89977ca from medications.csv
Added data for patient 19ac1829-945a-c4a3-63fb-ce50c89977ca from medications.csv
Added data for patient 19ac1829-945a-c4a3-63fb-ce50c89977ca from medications.csv
Added data for patient 19ac1829-945a-c4a3-63fb-ce50c89977c

KeyboardInterrupt: 